In [3]:
import pandas as pd
import json
import datetime

In [4]:
def decode_v(x):
    r = x[1:-1]
    r = r.replace("'",'"')
    s = r.split(': ')[0]+': '
    r = r.split(s)[1:]
    r = [s+i for i in r]
    r = [i[:-2] if i[-2:]==', ' else i for i in r]
    r = [json.loads(i) for i in r]
    return r

def ts2dt(x):
    return datetime.datetime.fromtimestamp(x)


def get_vout(block_path):
    d = pd.read_csv(block_path)
    d['vout'] = d['vout'].map(decode_v)

    t = enumerate(d['vout'].values)
    t = [{**{'index':i},**j} for (i,v) in t for j in v]
    t = pd.DataFrame.from_dict(t)
    
    vout = t['scriptPubKey'].values
    vout = pd.DataFrame.from_records(vout)
    vout = pd.concat([t.iloc[:,:2], vout], axis=1)
    vout = pd.merge(vout,d.reset_index(), how='left', on='index')
    vout['time'] = vout['time'].map(ts2dt)
    vout['date'] = vout['time'].dt.date
    return vout



def get_vin(block_path):
    d = pd.read_csv(block_path)
    d['vin'] = d['vin'].map(decode_v)

    t = enumerate(d['vin'].values)
    t = [{**{'index':i},**j} for (i,v) in t for j in v]
    t = pd.DataFrame.from_dict(t)

    vin = t['scriptSig'].map(lambda x: x if x==x else {'asm': None, 'hex': None})#.values
    vin = pd.DataFrame.from_records(vin)
    vin = pd.concat([t.iloc[:,:2], vin], axis=1)
    vin = pd.merge(vin,d.reset_index(), how='left', on='index')

    vin['time'] = vin['time'].map(ts2dt)
    vin['date'] = vin['time'].dt.date

    return vin


In [5]:
def filter_tx(tx, check_path):
    check = pd.read_csv(check_path, index_col=0)
    check['ltime'] = pd.to_datetime(check['ltime'])
    check['date'] = check['ltime'].dt.date
    
    tmp = pd.merge(left=tx, right=check, how='inner', on='date')
    
    tmp['delta_time'] = (tmp['time'] - tmp['ltime']) / datetime.timedelta(minutes=1)
    tmp = tmp[tmp['delta_time'].abs()<30]
    tmp = tmp[tmp['delta_time']>0]
    tmp['delta_sum'] = 2*(tmp['value'] - tmp['btcsum']) / (tmp['btcsum'] + tmp['value'])
    tmp = tmp[tmp['delta_sum'] < 0.1]
    tmp = tmp[tmp['delta_sum'] > 0]

    return tmp

In [6]:
current_block_csv = './rpc_block_834577_txs.csv'
check_csv = './check_btc.csv'

filter_tx(get_vout(current_block_csv),check_csv)

,index,value,asm,desc,hex_x,address,type,in_active_chain,txid,hash,...,hex_y,blockhash,confirmations,time,blocktime,date,ltime,btcsum,delta_time,delta_sum


In [7]:
f = pd.read_csv('./filtered_rpc_vouts2.csv').reset_index()
f['ltime'].unique()

array(['2024-03-13 22:35:25', '2024-03-13 21:50:27',
       '2024-03-13 22:06:34', '2024-03-13 22:08:39',
       '2024-03-14 11:21:29', '2024-03-14 11:36:48',
       '2024-03-16 09:45:33', '2024-03-17 07:52:21'], dtype=object)

In [8]:
cols = ['value', 'asm', 'address', 'txid', 'hash','vin', 'vout', 'blockhash', 'blocktime',
       'blockNumber','ltime', 'btcsum', 'delta_time', 'delta_sum']
#f[cols].to_csv('./bitcoin_nw_res1_raw.csv')

In [21]:
btcl = ['filtered_rpc_vouts_835530_835536_2024_03_20_11_43_54.csv',
        'filtered_rpc_vouts_835542_835548_2024_03_20_14_22_59.csv',
        'filtered_rpc_vouts_836104_836110_2024_03_24_13_52_48.csv',
        'filtered_rpc_vouts_836109_836115_2024_03_24_14_24_07.csv',
        'filtered_rpc_vouts_836242_836248_2024_03_25_13_31_22.csv']

d = ['./pb 2024-06-19/'+btcl[i] for i in range(5)]
d = [pd.read_csv(i) for i in d]

[(i['time'].min(),
  i['time'].max()) for i in d]

[('2024-03-20 11:14:41', '2024-03-20 11:15:53'),
 ('2024-03-20 13:33:43', '2024-03-20 13:33:43'),
 (nan, nan),
 ('2024-03-24 13:56:02', '2024-03-24 13:56:02'),
 ('2024-03-25 12:03:39', '2024-03-25 12:03:39')]

In [28]:
d = ['./pb 2024-06-19/'+btcl[i] for i in range(5)]
d = [pd.read_csv(i) for i in d]
d = pd.concat(d)
d['vout'].map(decode_v).map(lambda x: sum([i['value']==0.000713040 for i in x])).sum()
d['txid'].isin(['b8bea467a95c2a2358540190f6654856d11412e8304e68714a88bc430cd3beff']).sum()

/tmp/ipykernel_336273/3461870833.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  d = pd.concat(d)


0

In [29]:
d.to_csv('./pb 2024-06-19/res0_raw.csv')

In [38]:
d[['index','value','vin','vout','btcsum','delta_sum']]

,index,value,vin,vout,btcsum,delta_sum
0,17,0.000266,[{'txid': '837e15ce0dd72eb3d3e2062d1b62dbde477...,"[{'value': 0.03079509, 'n': 0, 'scriptPubKey':...",0.000257,0.035504
1,27,0.000344,[{'txid': 'bcc8939c28df09d49a533444952f45d1a27...,"[{'value': 0.00158392, 'n': 0, 'scriptPubKey':...",0.000325,0.055600
2,78,0.000345,[{'txid': '00ebdfc6374595a54c682d6f9ac808cec92...,"[{'value': 0.0, 'n': 0, 'scriptPubKey': {'asm'...",0.000325,0.058734
3,89,0.000053,[{'txid': '428456007e50efb130c0b7e12046c089b61...,"[{'value': 0.00136242, 'n': 0, 'scriptPubKey':...",0.000051,0.033174
4,89,0.000053,[{'txid': '428456007e50efb130c0b7e12046c089b61...,"[{'value': 0.00136242, 'n': 0, 'scriptPubKey':...",0.000051,0.033174
...,...,...,...,...,...,...
46,898,0.000169,[{'txid': '4b1a86b3aa5a501c9b9909c6994beb23d18...,"[{'value': 0.00060324, 'n': 0, 'scriptPubKey':...",0.000158,0.067762
47,921,0.000159,[{'txid': '1ed1092254df98c0b717442f5ee3126f47b...,"[{'value': 9.058e-05, 'n': 0, 'scriptPubKey': ...",0.000158,0.006101
48,979,0.000167,[{'txid': 'db36bc06bb09b87b66983cf1ee27f1ad022...,"[{'value': 4.488e-05, 'n': 0, 'scriptPubKey': ...",0.000158,0.057293
49,983,0.000211,[{'txid': '7274893cdb994f3f9b6be1699fabe15619b...,"[{'value': 0.0001432, 'n': 0, 'scriptPubKey': ...",0.000195,0.080019


In [42]:
d['vout'].map(decode_v).map(lambda x: [i['value'] for i in x])

0                            [0.03079509]
1                [0.00158392, 0.00296187]
2     [0.0, 0.00231, 0.00231, 4.41589236]
3                            [0.00136242]
4                            [0.00136242]
                     ...                 
46                           [0.00060324]
47                            [9.058e-05]
48                [4.488e-05, 0.00026913]
49                 [0.0001432, 0.0016886]
50                 [0.0001432, 0.0016886]
Name: vout, Length: 229, dtype: object